In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pydicom
import tensorflow as tf

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from keras.callbacks import CSVLogger
from livelossplot.keras import PlotLossesCallback

Using TensorFlow backend.


In [3]:
def load_images_from_folder(folder):
    images = []
    for i in range(1,601):
        filename = "Mask ("+str(i)+").tif"
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [4]:
masks = load_images_from_folder("mask")

In [5]:
def load_dicom_images_from_folder(folder):
    images = []
    for i in range(1,601):
        filename = "Image ("+str(i)+").dcm"
        data = pydicom.dcmread(os.path.join(folder,filename))
        img = data.pixel_array
        if img is not None:
            images.append(img)
    return images

In [6]:
images = load_dicom_images_from_folder("images")

In [ ]:
plt.imshow(images[179], cmap=plt.cm.bone)
plt.show()

In [7]:
maskg = []
for i in range(600):    
    maskg.append(cv2.cvtColor(masks[i], cv2.COLOR_BGR2GRAY))

In [8]:
idx = []
count = 0
for i in range(600):
    if(images[i].shape != maskg[i].shape):
        idx.append(i)
        count+=1
print(idx,count)

([], 0)


In [9]:
ildimg = []
for i in range(600): 
    if i not in idx:
        ildimg.append(np.multiply(images[i],maskg[i]))

In [10]:
def crop(img, tol=0):
    mask = img>tol
    m,n = img.shape
    mask0,mask1 = mask.any(0),mask.any(1)
    col_start,col_end = mask0.argmax(), n-mask0[::-1].argmax()
    row_start,row_end = mask1.argmax(),m-mask1[::-1].argmax()
    cropped = img[row_start:row_end,col_start:col_end]
    #final = img.reshape(cropped,(512,512))
    return cropped

In [11]:
croppi = []
for i in range (600):
    #print(i)
    a = crop(ildimg[i])
    croppi.append(a)
    
croppi = np.asarray(croppi)

In [12]:
final = []
for i in range(600):
    croppi[i] = np.asarray(croppi[i])
    final.append(cv2.resize(croppi[i],(28,28),interpolation=cv2.INTER_AREA))

In [13]:
final = np.asarray(final)
final.shape

(600, 28, 28)

In [14]:
target = []
for i in range(6):
    for j in range(i*100,(i+1)*100):
        target.append(i)
target = np.asarray(target)

In [15]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(final, target, test_size=0.2, random_state=10)
xtrain = np.asarray(xtrain)
ytrain = np.asarray(ytrain)
xtest = np.asarray(xtest)
ytest = np.asarray(ytest)

xtrain = xtrain.reshape(xtrain.shape[0], 256, 256,1)
xtest = xtest.reshape(xtest.shape[0], 256, 256,1)

xtrain = xtrain.astype('float32')
xtest = xtest.astype('float32')
xtrain /= 255
xtest /= 255

from keras.utils import np_utils
ytrain = np_utils.to_categorical(ytrain, 6)
ytest = np_utils.to_categorical(ytest, 6)
ytrain.shape

classifier = Sequential()

classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (256,256,1)))
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

classifier.add(Dense(units = 64, activation = 'relu'))

classifier.add(Dense(units = 6 , activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])

classifier.summary()

classifier.fit(xtrain,ytrain, epochs = 1, steps_per_epoch = 3)

In [16]:
from keras.utils import to_categorical

img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# Prepare the training images
xtrain = xtrain.reshape(xtrain.shape[0], img_rows, img_cols, 1)
xtrain = xtrain.astype('float32')
xtrain /= 255

# Prepare the test images
xtest = xtest.reshape(xtest.shape[0], img_rows, img_cols, 1)
xtest = xtest.astype('float32')
xtest /= 255


In [17]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(6, activation='softmax'))

cnn1.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
cnn1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 774       
Total params: 693,446
Trainable params: 693,446
Non-trainable params: 0
________________________________________________

In [19]:
cnn1.fit(xtrain,ytrain, epochs = 100, steps_per_epoch = 25)

Epoch 1/100
25/25 [==============================] - 5s 197ms/step - loss: 19.9422 - accuracy: 0.2455
Epoch 2/100
25/25 [==============================] - 6s 235ms/step - loss: 1.5840 - accuracy: 0.2783
Epoch 3/100
25/25 [==============================] - 5s 181ms/step - loss: 1.4475 - accuracy: 0.3394
Epoch 4/100
25/25 [==============================] - 4s 169ms/step - loss: 1.3148 - accuracy: 0.3848
Epoch 5/100
25/25 [==============================] - 4s 176ms/step - loss: 1.2228 - accuracy: 0.4388
Epoch 6/100
25/25 [==============================] - 4s 163ms/step - loss: 1.0683 - accuracy: 0.5358
Epoch 7/100
25/25 [==============================] - 4s 167ms/step - loss: 0.9333 - accuracy: 0.6020
Epoch 8/100
25/25 [==============================] - 4s 170ms/step - loss: 0.8273 - accuracy: 0.6494
Epoch 9/100
25/25 [==============================] - 4s 179ms/step - loss: 0.7092 - accuracy: 0.7108
Epoch 10/100
25/25 [==============================] - 4s 173ms/step - loss: 0.5895 - accur

In [20]:
xtrain.shape

(480, 28, 28, 1)

In [21]:
score1 = cnn1.evaluate(xtest, ytest, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

('Test loss:', 9.321265602111817)
('Test accuracy:', 0.4333333373069763)
